### Silver Layer: Process drivers data

Read from bronze/raw layer (parquet) and transform for silver/processed layer.

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from formula1.formula1_constants import raw_folder_path, processed_folder_path

In [ ]:
from formula1.formula1_utils import add_ingestion_date

##### Step 1 - Read parquet from bronze/raw layer

In [ ]:
drivers_df = spark.read.parquet(f"{raw_folder_path}/drivers")

##### Step 2 - Rename columns and add new columns
1. driverId renamed to driver_id  
1. driverRef renamed to driver_ref  
1. ingestion date updated
1. name added with concatenation of forename and surname

In [ ]:
from pyspark.sql.functions import col, concat, lit

In [ ]:
drivers_with_ingestion_date_df = add_ingestion_date(drivers_df)

In [ ]:
drivers_with_columns_df = drivers_with_ingestion_date_df \
    .withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("driverRef", "driver_ref") \
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

##### Step 3 - Drop the unwanted columns
1. url

In [ ]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

##### Step 4 - Write to output to silver/processed layer in parquet format

In [ ]:
drivers_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")